LSTM
---

Используя файл Web5.ipynb (он в полезных ссылках):

Приложен ноутбук, в котором реализованы функции для генерации из большого датасета в меньшую его копию. Вам нужно сделать следующее:

1. Сгенерировать меньший датасет из 8-10 классов движения
2. Обучить уже существующую модель (предварительно проанализировав, какие параметры модели нужно изменить)
3. Изменить модель: посмотреть зависимость от количества LSTM модулей в модели
4. *Сгенерировать другой датасет с меньшим количеством “кадров” в серии и сравнить улучшилось или ухудшилось качество предсказания. Провести несколько таких итераций, дать свою оценку уменьшению и увеличению кадров, назвать оптимальное, на ваш взгляд, их количество.

ДЗ в процессе выполнения...